<a href="https://colab.research.google.com/github/hellocybernetics/TensorFlow_Eager_Execution_Tutorials/blob/master/tutorials/02_intermediate/Bidrectional_Recurrent_Neural_Network_using_KerasAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tf-nightly-2.0-preview

In [0]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

L = tf.keras.layers

In [3]:
# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 512
learning_rate = 0.001

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

print("training_data: ", x_train.shape)
print("test_data: ", x_test.shape)
print("training_label: ", y_train.shape)
print("test_label: ", y_test.shape)

training_data:  (60000, 28, 28)
test_data:  (10000, 28, 28)
training_label:  (60000,)
test_label:  (10000,)


In [4]:
x_train_eager = tf.convert_to_tensor(x_train, dtype=tf.float32)
x_test_eager = tf.convert_to_tensor(x_test, dtype=tf.float32)
y_train_eager = tf.reshape(tf.one_hot(y_train, 10), (-1, 10))
y_test_eager = tf.reshape(tf.one_hot(y_test, 10), (-1, 10))

print("training_data: ", x_train_eager.shape)
print("test_data: ", x_test_eager.shape)
print("training_label: ", y_train_eager.shape)
print("test_label: ", y_test_eager.shape)

training_data:  (60000, 28, 28)
test_data:  (10000, 28, 28)
training_label:  (60000, 10)
test_label:  (10000, 10)


### DataSet
You make Dataset using `tf.data.Dataset` Class but Keras API doesn't need this dataset. If you write training loop code manually, `Dataset` class is very useful. And using keras API, you need numpy.array inputs instead of tf.Tensor. I don't know why...so you only need numpy preprocessing (or get numpy.array from tf.Tensor using numpy() method after preprocessing using function of tf).

### NOTE
This notebook we don't need 'tf.data.Dataset'. This code only just for reference.

In [0]:
train_dataset = (
    tf.data.Dataset.from_tensor_slices((x_train_eager, y_train_eager))
    .batch(batch_size)
    .shuffle(10000)
)
train_dataset = train_dataset.repeat()

In [0]:
test_dataset = (
    tf.data.Dataset.from_tensor_slices((x_test_eager, y_test_eager))
    .batch(1000)
    .shuffle(10000)
)
test_dataset = test_dataset.repeat()

### RNN using LSTM
In keras API, LSTM recives inputs tensor whose shape is (batch_size, seqence_length, feature_dim), and output tensor whose shape is (batch_size, fearure_dim).When you need all time sequence data, you have to give `return_sequences=True` to LSTM's constractor. Generally, when you stack LSTM's, you need all sequence data.

We use  just only `tf.keras.layers.Bidirectional` for using Bidrectional LSTM.

In [0]:
class RNN(tf.keras.Model):
    def __init__(self, hidden_size=10, num_layers=2, num_classes=10):
        super(RNN, self).__init__(name='mnist_rnn')
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = self.get_lstm_layers(hidden_size, num_layers)            
        self.fc = L.Dense(num_classes, activation="softmax")
    
    @staticmethod
    def get_lstm_layers(hidden_size, num_layers):
        lstm_layers = []
        # we need all sequence data. write return_sequences=True! 
        for i in range(num_layers-1):
            lstm_layers.append(
                L.Bidirectional(
                    L.LSTM(units=hidden_size, 
                                         return_sequences=True)
                )
            )
        # the final layer return only final sequence
        # if you need all sequences, you have to write return_sequences=True.
        lstm_layers.append(
            L.Bidirectional(
                L.LSTM(units=hidden_size)
            )
        )
        return tf.keras.Sequential(lstm_layers)
        
    def call(self, x):        
        # Forward propagate LSTM
        out = self.lstm(x)
        out = self.fc(out)
        return out

In [0]:
model = RNN()

In [9]:
optimizer = tf.keras.optimizers.Adam(learning_rate)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=["accuracy"])

# Eager Execution initialize parameters when using model.call()
model(x_train_eager[:50])

model.summary()

Model: "mnist_rnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      multiple                  5600      
_________________________________________________________________
dense (Dense)                multiple                  210       
Total params: 5,810
Trainable params: 5,810
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.fit(x=x_train_eager.numpy(), 
          y=y_train_eager.numpy(), 
          validation_split=0.2, 
          epochs=num_epochs,
          batch_size=batch_size)

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000==============================] - 12s 254us/sample - loss: 2.0411 - acc: 0.3131 - val_loss: 1.5728 - val_acc: 0.5153
Epoch 2/5
48000/48000==============================] - 10s 215us/sample - loss: 1.2057 - acc: 0.6374 - val_loss: 0.8727 - val_acc: 0.7474
Epoch 3/5
48000/48000==============================] - 10s 216us/sample - loss: 0.7447 - acc: 0.7834 - val_loss: 0.6034 - val_acc: 0.8220
Epoch 4/5
48000/48000==============================] - 10s 214us/sample - loss: 0.5666 - acc: 0.8342 - val_loss: 0.4935 - val_acc: 0.8529
Epoch 5/5
48000/48000==============================] - 10s 215us/sample - loss: 0.4756 - acc: 0.8593 - val_loss: 0.4198 - val_acc: 0.8751


In [11]:
test_loss, test_acc = model.evaluate(x=x_test_eager.numpy(), 
                                     y=y_test_eager.numpy())

print("test_accracy: ", test_acc)

10000/10000==============================] - 2s 240us/sample - loss: 0.4305 - acc: 0.8709
test_accracy:  0.8709
